Here is an example data of 10 field sites from 5 studies.  
There are base tillage treatments ('cp', 'dr', 'mp', 'cp_cc', 'mp_cc') and interventions ('nt', 'st', 'nt_cc').

In [1]:
source("cov_matrix.r")
example_data <- read.csv("vmd53_example_data.csv")
colnames(example_data)
unique(example_data$study)
unique(example_data$location)
head(example_data)

[1] "doi"           "study"         "auth"          "treatment"    
 [5] "trt_yr_st"     "trt_yr_end"    "yrs_btwn_meas" "location"     
 [9] "site_id"       "rotation"      "commodity"     "output"       
[13] "value"         "yrly_delta"    "diff_delta"    "unit"         
[17] "soc_interp"    "std"           "lyr_top"       "lyr_btm"      
[21] "repl_measures" "rep_meas"

[1] "AlKaisi_2020" "Schmer_2014"  "Olson_2010"   "Olson_2005"   "Sanford_2012"

[1] "Sutherland"     "Kanawha"        "Nashua"         "Ames"          
 [5] "Armstrong"      "McNay"          "Crawfordsville" "Ithaca"        
 [9] "Dixon"          "WICST"

,doi,study,auth,treatment,trt_yr_st,trt_yr_end,yrs_btwn_meas,location,site_id,rotation,⋯,value,yrly_delta,diff_delta,unit,soc_interp,std,lyr_top,lyr_btm,repl_measures,rep_meas
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,nt,2002,2007,5,Sutherland,9,cs,⋯,2.15,0.430,0.7431667,Mg ha-1,delta,na,0,60,40,2
2,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,st,2002,2007,5,Sutherland,9,cs,⋯,1.80,0.360,0.7431667,Mg ha-1,delta,na,0,60,40,2
3,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,cp,2002,2007,5,Sutherland,9,cs,⋯,-1.86,-0.372,0.7431667,Mg ha-1,delta,na,0,60,40,2
4,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,dr,2002,2007,5,Sutherland,9,cs,⋯,-1.62,-0.324,0.7431667,Mg ha-1,delta,na,0,60,40,2
5,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,mp,2002,2007,5,Sutherland,9,cs,⋯,-2.04,-0.408,0.7431667,Mg ha-1,delta,na,0,60,40,2
6,10.1002/saj2.20003,AlKaisi_2020,Al-Kaisi,nt,2002,2007,5,Sutherland,9,ccs,⋯,2.11,0.422,0.6665833,Mg ha-1,delta,na,0,60,40,2


I have included features as identified from the Mathers cov matrix plus some.    
Below is one of the fields. I have rep measures as 2 since there are repeat measures  
at two different time points.

In [2]:
my_cols <- c("rep_meas", "treatment", "rotation", "trt_yr_end", "location", "yrs_btwn_meas")

In [3]:
suth_data <- subset(example_data, location == 'Sutherland')
suth_data[, my_cols]

,rep_meas,treatment,rotation,trt_yr_end,location,yrs_btwn_meas
,<int>,<chr>,<chr>,<int>,<chr>,<int>
1,2,nt,cs,2007,Sutherland,5
2,2,st,cs,2007,Sutherland,5
3,2,cp,cs,2007,Sutherland,5
4,2,dr,cs,2007,Sutherland,5
5,2,mp,cs,2007,Sutherland,5
6,2,nt,ccs,2007,Sutherland,5
7,2,st,ccs,2007,Sutherland,5
8,2,cp,ccs,2007,Sutherland,5
9,2,dr,ccs,2007,Sutherland,5


Here's another location from which we have data from two separate papers,  
so across both papers there are 9 repeated measures.

In [4]:
dixon_data <- subset(example_data, location == 'Dixon')
head(dixon_data[, my_cols])

,rep_meas,treatment,rotation,trt_yr_end,location,yrs_btwn_meas
,<int>,<chr>,<chr>,<int>,<chr>,<int>
186,9,nt,cs,2003,Dixon,3
187,9,nt,cs,2007,Dixon,4
188,9,nt,cs,2009,Dixon,2
189,9,nt_cc,cs,2003,Dixon,3
190,9,nt_cc,cs,2007,Dixon,4
191,9,nt_cc,cs,2009,Dixon,2


Let's see if this looks good. Construct a cov matrix just for the corn-soy rotation observations.  
note: this rotation could quality for 'crop functional group' soybean (C3 plant) or maize (C4 plant),  
but other rotations (e.g. corn-corn) would only qualify for the C4 group.

In [5]:
corn_soy_data <- subset(example_data, rotation == 'cs')

In [6]:
constr_mather_cov <- function(params, site_ids, obs_years, yrs_btwn_meas) {
  # Site random effect
  construct_random_effect_covariance(
    params[1],
    site_ids) +

    # Year-within-site random effect
    construct_random_effect_covariance(
      params[2],
      paste0(
        site_ids,
        obs_years)) +

    # Time-based residual variance
    construct_temporal_independent_covariance(
      params[3],
      params[4],
      yrs_btwn_meas
    )
}

In [7]:
params_df <- read.csv("calib_params.csv")
cov_params <- tail(params_df, 4)


In [8]:
params <- log(cov_params$default^2)
example_cov_matrix <- constr_mather_cov(
    params,
    corn_soy_data$site_id,
    corn_soy_data$trt_yr_end,
    corn_soy_data$yrs_btwn_meas
)
example_cov_matrix[1:10,]

,1,2,3,4,5,6,7,8,9,10,⋯,98,99,100,101,102,103,104,105,106,107
1,0.0172023,0.0136000,0.0136000,0.0136000,0.0136000,0.00360000,0.00360000,0.00360000,0.00360000,0.00360000,⋯,0,0,0,0,0,0,0,0,0,0
2,0.0136000,0.0172023,0.0136000,0.0136000,0.0136000,0.00360000,0.00360000,0.00360000,0.00360000,0.00360000,⋯,0,0,0,0,0,0,0,0,0,0
3,0.0136000,0.0136000,0.0172023,0.0136000,0.0136000,0.00360000,0.00360000,0.00360000,0.00360000,0.00360000,⋯,0,0,0,0,0,0,0,0,0,0
4,0.0136000,0.0136000,0.0136000,0.0172023,0.0136000,0.00360000,0.00360000,0.00360000,0.00360000,0.00360000,⋯,0,0,0,0,0,0,0,0,0,0
5,0.0136000,0.0136000,0.0136000,0.0136000,0.0172023,0.00360000,0.00360000,0.00360000,0.00360000,0.00360000,⋯,0,0,0,0,0,0,0,0,0,0
6,0.0036000,0.0036000,0.0036000,0.0036000,0.0036000,0.01720323,0.01360000,0.01360000,0.01360000,0.01360000,⋯,0,0,0,0,0,0,0,0,0,0
7,0.0036000,0.0036000,0.0036000,0.0036000,0.0036000,0.01360000,0.01720323,0.01360000,0.01360000,0.01360000,⋯,0,0,0,0,0,0,0,0,0,0
8,0.0036000,0.0036000,0.0036000,0.0036000,0.0036000,0.01360000,0.01360000,0.01720323,0.01360000,0.01360000,⋯,0,0,0,0,0,0,0,0,0,0
9,0.0036000,0.0036000,0.0036000,0.0036000,0.0036000,0.01360000,0.01360000,0.01360000,0.01720323,0.01360000,⋯,0,0,0,0,0,0,0,0,0,0
10,0.0036000,0.0036000,0.0036000,0.0036000,0.0036000,0.01360000,0.01360000,0.01360000,0.01360000,0.01720323,⋯,0,0,0,0,0,0,0,0,0,0
